# Neural translation machine from hindi-english language open source corpouas from statsloc.pou
https://www.kaggle.com/aiswaryaramachandran/english-to-hindi-neural-machine-translation

In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

C:\Users\Darshit\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:27: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [2]:
lines=pd.read_csv("E:/kaggle_dataset/hindi_english_parallel/hindi_english_parallel.csv",encoding='utf-8')

In [3]:
lines.head(10)

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default
5,अवधि को हाइलाइट रकें,Highlight duration
6,पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्स...,The duration of the highlight box when selecti...
7,सीमांत (बोर्डर) के रंग को हाइलाइट करें,Highlight border color
8,हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता।,The color and opacity of the highlight border.
9,भराई के रंग को हाइलाइट करें,Highlight fill color


In [19]:
lines=lines[~pd.isnull(lines['english'])]
lines=lines[~pd.isnull(lines['hindi'])]
lines.drop_duplicates(inplace=True)

In [23]:
lines=lines.sample(n=250,random_state=42)# sample n=2500
lines.shape

(250, 2)

In [24]:
# Lowercase all characters
lines['english']=lines['english'].apply(lambda x: x.lower())
lines['hindi']=lines['hindi'].apply(lambda x: x.lower())

In [25]:
import re
# Remove quotes
lines['english']=lines['english'].apply(lambda x: re.sub("'", '', x))
lines['hindi']=lines['hindi'].apply(lambda x: re.sub("'", '', x))

In [27]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english']=lines['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi']=lines['hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [28]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english']=lines['english'].apply(lambda x: x.translate(remove_digits))
lines['hindi']=lines['hindi'].apply(lambda x: x.translate(remove_digits))

lines['hindi'] = lines['hindi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english']=lines['english'].apply(lambda x: x.strip())
lines['hindi']=lines['hindi'].apply(lambda x: x.strip())
lines['english']=lines['english'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi']=lines['hindi'].apply(lambda x: re.sub(" +", " ", x))

In [29]:
# Add start and end tokens to target sequences
lines['hindi'] = lines['hindi'].apply(lambda x : 'START_ '+ x + ' _END')

In [30]:
lines.head()

,hindi,english
1089912,START_ लेकिन इन तीरों आकर्षित करने के लिए सबसे आसान तरीका है मैं हमेशा करता है और _END,but the easiest way i always do is to draw these arrows and
307295,START_ कह दो इसे ता पवित्र आत्मा ने तुम्हारे रब की ओर क्रमशः सत्य के साथ उतारा है ताकि ईमान लानेवालों को जमाव प्रदान करे और आज्ञाकारियों के लिए मार्गदर्शन और शुभ सूचना हो _END,say “the holy spirit has brought it down from your lord truthfully in order to stabilize those who believe and as guidance and good news for those who submit ”
654271,START_ जिसके पास कैंसर का इलाज है _END,who had the cure for cancer
1221838,START_ ‘‘बहुत सारी मशीनरी और ऐसी ही वस्तुओं के प्रयोग से भौतिक जीवन के सुखों को बढ़ाने में सफलता प्राप्त करने से ही कोई राष्ट्र सभ्य नहीं बन सकता इस युग में जहां एक ओर लोगों को अत्यंत व्यावहारिक बना होगा वहीं दूसरी ओर उन्हें गहरा आध्यात्मिक ज्ञान प्राप्त करना होगा। ’’ _END,no nation can be said to have become civilized only because it has succeeded in increasing the comforts of material life by bringing into use lots of machinery and things of that sort in this age as on the one hand people have to be intensely practical so on the other they have to acquire deep spiritual knowledge
1005692,START_ समुचित या पर्याप्त मात्रा में। _END,something that is adequate or sufficient in quantity


In [31]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [32]:
len(all_eng_words)

1337

In [33]:
len(all_hindi_words)

1526

In [34]:
lines['length_eng_sentence']=lines['english'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi'].apply(lambda x:len(x.split(" ")))

In [35]:
lines.head()

,hindi,english,length_eng_sentence,length_hin_sentence
1089912,START_ लेकिन इन तीरों आकर्षित करने के लिए सबसे आसान तरीका है मैं हमेशा करता है और _END,but the easiest way i always do is to draw these arrows and,13,18
307295,START_ कह दो इसे ता पवित्र आत्मा ने तुम्हारे रब की ओर क्रमशः सत्य के साथ उतारा है ताकि ईमान लानेवालों को जमाव प्रदान करे और आज्ञाकारियों के लिए मार्गदर्शन और शुभ सूचना हो _END,say “the holy spirit has brought it down from your lord truthfully in order to stabilize those who believe and as guidance and good news for those who submit ”,30,35
654271,START_ जिसके पास कैंसर का इलाज है _END,who had the cure for cancer,6,8
1221838,START_ ‘‘बहुत सारी मशीनरी और ऐसी ही वस्तुओं के प्रयोग से भौतिक जीवन के सुखों को बढ़ाने में सफलता प्राप्त करने से ही कोई राष्ट्र सभ्य नहीं बन सकता इस युग में जहां एक ओर लोगों को अत्यंत व्यावहारिक बना होगा वहीं दूसरी ओर उन्हें गहरा आध्यात्मिक ज्ञान प्राप्त करना होगा। ’’ _END,no nation can be said to have become civilized only because it has succeeded in increasing the comforts of material life by bringing into use lots of machinery and things of that sort in this age as on the one hand people have to be intensely practical so on the other they have to acquire deep spiritual knowledge,58,53
1005692,START_ समुचित या पर्याप्त मात्रा में। _END,something that is adequate or sufficient in quantity,8,7


In [36]:
lines[lines['length_eng_sentence']>30].shape

(21, 4)

In [37]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [38]:
lines.shape

(184, 4)

In [39]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  19


In [40]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])


In [41]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(1337, 1526)

In [42]:
num_decoder_tokens += 1 #for zero padding

In [43]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [44]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [45]:
lines = shuffle(lines)
lines.head(10)

,hindi,english,length_eng_sentence,length_hin_sentence
329620,START_ बेशक ये कुरान सरासर नसीहत है तो जो शख़्श चाहे अपने परवरदिगार की राह ले _END,verily this is an exhortation so let him who so will take a way to his lord,17,17
1335255,START_ एक स्थान पर वह अखंड कावेरी है सहायक नदियों के जल से उफनती विशाल नदी। _END,at one place it is akhanda kaveri the immense river swollen with tributaries,13,17
998343,START_ कर लगे लाभ पर कर पुनः कर लगाना दोहरा कर लगाना होगा। _END,taxing taxed profits would amount to double taxation,8,14
313866,START_ और यहूदी लोग कहते हैं कि खुदा ने अज़ीज़ को बेटा बना लिया है _END,and they say “the most merciful has begotten a son ”,11,16
1199393,START_ रेलवे सूचना प्रणाली केंद्र _END,centre for railway information systems,5,6
1203418,START_ भारत सरकार और भारतीय रिजर्व बैंक द्वारा समितियों के साथ सहयोग हाल के वर्ष _END,association with committees by goi and rbi recent years,9,16
990168,START_ उपचार के बाद ही इस कि पुष्टि की गई कि वह बहुतंत्रिकार्ति से पीड़ित था _END,only after diagonisis it was confirmed that he was suffering from polyneuralgia,12,17
222652,START_ केवर्ड एमिप्रो निर्यात फ़िल्टरname _END,kword amipro export filter,4,6
964895,START_ वह नीचे अमेज़न नदी में छोटी नाव से यात्रा करने गया। _END,he went canoeing down the amazon river once,8,13
506386,START_ परछाईयाँ उन्हे डरा देती थी एक ज़मीन पर पडा पाईप भी _END,shadows would make them balk a hose on the floor,10,13


In [46]:
# spliting of data

X, y = lines['english'], lines['hindi']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((147,), (37,))

In [47]:
X_train.to_pickle('X_trainenhi.pkl')
X_test.to_pickle('X_testenhi.pkl')

In [48]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [49]:
#Encoder-Decoder Architecture
latent_dim=300

In [50]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [51]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [52]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [53]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    401100      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    458100      input_2[0][0]                    
______________________________________________________________________________________________

In [61]:
train_samples = len(X_train)  #batch_size can give u the log error so accoring to ur data mention ur smaple in batch_size
val_samples = len(X_test)
batch_size = 10
epochs = 100

In [62]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/100
14/14 [==============================] - 17s 1s/step - loss: 2.5012 - val_loss: 2.6672
Epoch 2/100
14/14 [==============================] - 4s 266ms/step - loss: 2.3486 - val_loss: 2.7217
Epoch 3/100
14/14 [==============================] - 4s 263ms/step - loss: 2.1948 - val_loss: 2.8148
Epoch 4/100
14/14 [==============================] - 4s 261ms/step - loss: 2.1366 - val_loss: 2.9404
Epoch 5/100
14/14 [==============================] - 4s 260ms/step - loss: 2.0938 - val_loss: 3.0260
Epoch 6/100
14/14 [==============================] - 4s 261ms/step - loss: 2.0538 - val_loss: 3.0974
Epoch 7/100
14/14 [==============================] - 4s 264ms/step - loss: 1.9487 - val_loss: 3.1676
Epoch 8/100
14/14 [==============================] - 4s 260ms/step - loss: 1.9903 - val_loss: 3.2366
Epoch 9/100
14/14 [==============================] - 4s 262ms/step - loss: 1.8628 - val_loss: 3.3119
Epoch 10/100
14/14 [==============================] - 4s 272ms/step - loss: 1.8221 - val_loss

14/14 [==============================] - 3s 250ms/step - loss: 0.0036 - val_loss: 4.6123
Epoch 82/100
14/14 [==============================] - 4s 251ms/step - loss: 0.0026 - val_loss: 4.6124
Epoch 83/100
14/14 [==============================] - 3s 250ms/step - loss: 0.0017 - val_loss: 4.6487
Epoch 84/100
14/14 [==============================] - 4s 250ms/step - loss: 0.0013 - val_loss: 4.6506
Epoch 85/100
14/14 [==============================] - 4s 251ms/step - loss: 0.0011 - val_loss: 4.6746
Epoch 86/100
14/14 [==============================] - 4s 254ms/step - loss: 0.0012 - val_loss: 4.6276
Epoch 87/100
14/14 [==============================] - 3s 249ms/step - loss: 0.0023 - val_loss: 4.7411
Epoch 88/100
14/14 [==============================] - 4s 251ms/step - loss: 0.0023 - val_loss: 4.7124
Epoch 89/100
14/14 [==============================] - 4s 268ms/step - loss: 6.1930e-04 - val_loss: 4.7297
Epoch 90/100
14/14 [==============================] - 4s 250ms/step - loss: 4.9143e-04 - va

In [63]:
model.save_weights('nmthien_weights.h5')

In [64]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [65]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [66]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [71]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: president attends the th annual convocation of iit kanpur rashtrapati bhavan
Actual Hindi Translation:  राष्ट्रपति जी ने भारतीय प्रौद्योगिकी संस्थानकानपुर के वें दीक्षांत समारोह में भाग लिया। राष्ट्रपति भवन 
Predicted Hindi Translation:  राष्ट्रपति जी ने भारतीय प्रौद्योगिकी संस्थानका


In [72]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: the extenuating factor may help reforming a person
Actual Hindi Translation:  परिशमनकारी तत्व किसी व्यक्ति के सुधार में सहायक हो सकते हैं। 
Predicted Hindi Translation:  परिशमनकारी तत्व किसी व्यक्ति के सुधार में सहाय
